In [ ]:
import tensorflow.compat.v1 as tf
# #Lets see if tensorflow finds the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#import tensorflow as tf

In [ ]:
#Lets see if it works
tf.ones(1) + tf.ones(1)

In [ ]:
import numpy as np # for working with arrays and matrices
import pandas as pd # for data manipulation and analysis
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for data visualization
import time # for time-related functions
import random # for random number generation
import cv2 # for computer vision and image processing tasks
import datetime # for saving date and time information

import csv


import h5py # for working with HDF5 (Hierarchical Data Format) files
import boto3 # for working with Amazon Web Services (AWS)
from pynwb import NWBHDF5IO # for working with Neurodata Without Border (NWB) files
import fsspec 
from fsspec.implementations.cached import CachingFileSystem # library used for working with various file systems in Python.
import requests 
import aiohttp # libraries which are used for making HTTP requests in Python.
import os # OS module provides various operating system-related functions to the code
# import csv # CSV module is used for working with CSV (Comma Separated Values) files in Python.
import pickle
import re


# used for splitting data into training and testing sets in Python.
from sklearn.model_selection import train_test_split 

# for generating a confusion matrix
from sklearn.metrics import confusion_matrix



# # Classes and functions from the Keras library which is used for building and training deep learning models in Python.
# from keras.models import load_model
# from keras.models import model_from_json
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dropout
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# These import the Adam optimizer class and various other classes from the TensorFlow Keras library 
# which is a high-level neural networks API used for building and training deep learning models in Python.
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
import imgaug.augmenters as iaa



# # Loading functions
from load_calcium_video import load_video_data
from pixel_values_normalization import normalize_video
from align_behavior_to_calcium import align_files
from class_balance import check_class_imbalance
from model_architecture import construct_model
from preprocessing_model import model_preprocessing
from run_model import model_execution
from save_model_info import save_training_info
# from set_s3_connection import generate_s3_url
from plots import plot_first_frames, plot_random_frames
from send_email_when_code_is_run import send_email
from class_balance import check_distribution_among_datasets


from urllib.parse import urlparse
from ann_visualizer.visualize import ann_viz
import importlib
import sys


In [ ]:
# Goal:
experiment_name = "Building base model, turning-type labels, 20% val - 80% train"
experiment_ID = '2.5'
%store experiment_ID
comment = "Building base model... "+str(experiment_name)
train_test_split_strategy = "20% val - 80% train"
name = 'BPNN_base_v2'
model_version = str(name)+'.1'
# experiment_version = ""

In [ ]:
# Define the name of the output directory
output_dir = "output_base"

# Check if the output directory already exists
if not os.path.exists(output_dir):
    # Create the output directory
    os.mkdir(output_dir)

    # Create the balance, accuracy, loss, and cm directories inside the output directory
    os.mkdir(os.path.join(output_dir, "balance"))
    os.mkdir(os.path.join(output_dir, "accuracy"))
    os.mkdir(os.path.join(output_dir, "loss"))
    os.mkdir(os.path.join(output_dir, "cm"))
    os.mkdir(os.path.join(output_dir, "architecture"))
    os.mkdir(os.path.join(output_dir, "pickles"))
else:
    print(f"The directory {output_dir} already exists.")

In [ ]:
s3_calcium_url = 'https://s3.ki.se/dmc-striatum-arrowmaze/processed-data/miniscope-recordings/export-to-nwb/animal3learnday11/20211028_181307_animal3learnday11.nwb?AWSAccessKeyId=5AMYRX4EUZ0MV0276K24&Signature=ZrARakl7KWzfF6vI2c3nx8uh%2FFo%3D&Expires=1681304429'

In [ ]:
# # Parse animal and learning day information
video_name = os.path.basename(os.path.dirname(urlparse(s3_calcium_url).path))
print(video_name) # Output: animal1learnday1

In [ ]:
video_data = load_video_data(s3_calcium_url)

In [ ]:
min_frame = np.min(video_data, axis=0)
video_data = video_data - min_frame

In [ ]:
images = normalize_video(video_data)

In [ ]:
# Check if the video has been loaded correctly and normalized
images[:1]

In [ ]:
# Determine the size of the calcium video dataset
num_of_frames = images.shape[0]
img_height = images.shape[1]
img_width = images.shape[2]
print("The number of video frames is ", num_of_frames, " and the frame dimensions (height x width) are: ", img_height, "X", img_width)

In [ ]:
# Preparing bonsai data file.
# CSV with additional data from the behavior box, such as reward deliveries. Also includes information needed for synchronizing the calcium and behavioral recordings.
bonsai_data = pd.read_csv('/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/tmaze_2021-10-28T18_13_23.csv', header=None)
# Segmentation of each frame into one behavior class.
df_behavior = pd.read_hdf('/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211028_181307_animal3learnday11.h5', 'per_frame')

In [ ]:
df_new_annotations, df_unique_states = align_files(bonsai_data, df_behavior)

In [ ]:
df_new_annotations_unique = df_new_annotations.unique()
df_new_annotations_unique

In [ ]:
class_counts, total_counts = check_class_imbalance(df_new_annotations, experiment_ID)

In [ ]:
# Training Parameters
channel_dimension = 1
labels = df_new_annotations
epochs = 20
batch_size = 32

In [ ]:
images.shape

In [ ]:
# # Split the data into training and validation sets
split_index = int(0.2 * len(images))  # Index to split data
val_images, train_images = images[:split_index], images[split_index:]
val_labels, train_labels = labels[:split_index], labels[split_index:]

In [ ]:
check_distribution_among_datasets(val_labels, experiment_ID, dataset_type = 'Validation_set')
check_distribution_among_datasets(train_labels, experiment_ID, dataset_type = 'Training_set')

In [ ]:
train_images, val_images, train_labels, val_labels, num_classes = model_preprocessing(train_images, val_images, train_labels, val_labels, df_new_annotations_unique)

In [ ]:
# # Plot the first 5 random images
plot_first_frames(train_images, train_labels)
plot_first_frames(val_images, val_labels)

In [ ]:
plot_random_frames(train_images, train_labels)
plot_random_frames(val_images, val_labels)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten


def base_model(num_classes, name, input_shape):
    
    model = Sequential(name=name)
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=num_classes, activation='softmax'))
    
    return model

In [ ]:
import tensorflow.compat.v1 as tf
import time # for time-related functions
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import csv # CSV module is used for working with CSV (Comma Separated Values) files in Python.
# from f1_score import f1_score
import os

def base_model_execution(base_params):    
    
    model = base_params['model']
    tf = base_params['tf']
    train_images = base_params['train_images']
    train_labels = base_params['train_labels']
    epochs = base_params['epochs']
    batch_size = base_params['batch_size']
    validation_data = base_params['validation_data']
    val_images = base_params['val_images']
    val_labels = base_params['val_labels']
    
    start_time = time.time()
    
    # f1_score = f1_score()
    
    print("Compiling model...")
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.legacy.Adam(), metrics=['accuracy'])
    
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
    
    
    print("Running model. Go grab a coffee or smth.")
    history = model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=validation_data, callbacks=[early_stopping]) 
    
    end_time = time.time()
    execution_time = end_time - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution time: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds")

    
    dir_name = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/V2/output/pickles"
    model_save_name = 'BPNN_base_model_v1.h5'
    print("Now saving model data to pickles. Please wait...")
    model.save(f"{dir_name}/{model_save_name}.png")
    
    dir_name_pickles = "src/V2/output/pickels"
    # Save the history object to a pickle file
    with open(os.path.join(dir_name, 'history.pkl'), 'wb') as f:
        pickle.dump(history.history, f)
    
    with open(os.path.join(dir_name, 'train_images.pkl'), 'wb') as f:
        pickle.dump(train_images, f)

    with open(os.path.join(dir_name, 'val_images.pkl'), 'wb') as f:
        pickle.dump(val_images, f)

    with open(os.path.join(dir_name, 'train_labels.pkl'), 'wb') as f:
        pickle.dump(train_labels, f)
        
    with open(os.path.join(dir_name, 'val_labels.pkl'), 'wb') as f:
        pickle.dump(val_labels, f)
        
    print("Done!")

    
    return history

In [ ]:
input_shape = (img_height, img_width, channel_dimension)
input_shape

In [ ]:
name = 'base_model'

In [ ]:
base_model = base_model(num_classes, name, input_shape)

In [ ]:
# dir_name = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/V2/output/architecture"
# plot_name = f"{name}_architecture"
# ann_viz(base_model, view=True, filename=plot_name, title="CNN — "+str(name)+" — Simple Architecture")
# plot_path = os.path.join(dir_name, f"{plot_name}.png")
# plot = plt.gcf()
# plot.savefig(plot_path, dpi=300, bbox_inches="tight")

In [ ]:
validation_data=(val_images, val_labels)

In [ ]:
base_params = {
    'model': base_model,
    'tf': tf,
    'train_images': train_images,
    'train_labels': train_labels,
    'epochs': epochs,
    'batch_size': batch_size,
    'validation_data': validation_data,
    'val_images': validation_data[0],
    'val_labels': validation_data[1]
}

In [ ]:
base_history = base_model_execution(base_params)

In [ ]:
save_training_info(base_model, base_history, video_name, comment, experiment_ID)